In [26]:
import re
import csv
from collections import defaultdict
from datetime import datetime, timedelta

# Regex untuk parsing log format (Nginx/Apache)
log_pattern = re.compile(
    r'(?P<ip>\S+) - (?P<user>\S+) \[(?P<time>.*?)\] "(?P<method>\S+) (?P<url>\S+) \S+" '
    r'(?P<status>\d+) (?P<size>\d+) "(?P<referer>.*?)" "(?P<agent>.*?)"'
)

# Regex untuk Deteksi Pola Serangan
regex_patterns = {
    "SQL Injection": r"(?:'|\"|;|\b(OR|AND)\b.*?=|--|\bUNION\b.*?\bSELECT\b|\bINSERT\b.*?\bINTO\b|\bUPDATE\b.*?\bSET\b|\bDELETE\b.*?\bFROM\b)",
    "XSS": r"(<script.*?>.*?</script>|javascript:|on\w+=['\"]?.*?['\"]?|<.*?\bon\w+.*?=.*?>)",
    "Directory Traversal": r"(\.\./|\.\.\\|/etc/passwd|/windows/win.ini|/system32/)",
    "Command Injection": r"(;|\||&&|`|\$\(.*?\)|\b(cat|wget|curl|ls|echo|whoami|id|uname)\b)",
    "Path Manipulation": r"(/\.\.|\\\.\.)",
}

# Threshold untuk Brute Force dan DoS
brute_force_threshold = 5  # Jumlah percobaan gagal dari IP yang sama
dos_threshold = 20          # Jumlah request dalam 1 menit untuk DoS
ip_request_times = defaultdict(list)  # Menyimpan waktu request per IP
ip_failed_attempts = defaultdict(int)  # Menyimpan jumlah status 4xx/5xx per IP


In [27]:
def detect_attack(ip, method, url, status, log_time):
    """
    Fungsi untuk mendeteksi pola serangan dari log.
    
    Parameters:
        ip (str): Alamat IP client.
        method (str): Metode request HTTP (GET, POST, dsb.).
        url (str): URL atau endpoint request.
        status (str): Status kode HTTP (200, 404, dsb.).
        log_time (str): Waktu dalam format "%d/%b/%Y:%H:%M:%S %z".
    
    Returns:
        str: Jenis serangan yang terdeteksi atau 'Normal' jika tidak ada serangan.
    """
    # 1. **SQL Injection Detection**
    if re.search(regex_patterns["SQL Injection"], url, re.IGNORECASE):
        return "SQL Injection Detected"

    # 2. **XSS Detection (Cross-Site Scripting)**
    elif re.search(regex_patterns["XSS"], url, re.IGNORECASE):
        return "XSS Attack Detected"

    # 3. **Directory Traversal Detection**
    elif re.search(regex_patterns["Directory Traversal"], url, re.IGNORECASE):
        return "Directory Traversal Detected"

    # 4. **Command Injection Detection**
    elif re.search(regex_patterns["Command Injection"], url, re.IGNORECASE):
        return "Command Injection Detected"

    # 5. **Path Manipulation**
    elif re.search(regex_patterns["Path Manipulation"], url, re.IGNORECASE):
        return "Path Manipulation Detected"

    # 6. **Brute Force Detection**
    # Jika request gagal (4xx/5xx), simpan IP sebagai percobaan gagal
    if status.startswith('4') or status.startswith('5'):
        ip_failed_attempts[ip] += 1
        if ip_failed_attempts[ip] >= brute_force_threshold:
            return f"Brute Force Attack Detected (Failed Attempts: {ip_failed_attempts[ip]})"

    # 7. **DoS Attack Detection**
    # Simpan waktu request dalam deque untuk setiap IP
    try:
        log_time_obj = datetime.strptime(log_time, "%d/%b/%Y:%H:%M:%S %z")
        ip_request_times[ip].append(log_time_obj)

        # Filter request yang terjadi dalam 1 menit terakhir
        one_minute_ago = log_time_obj - timedelta(minutes=1)
        recent_requests = [t for t in ip_request_times[ip] if t > one_minute_ago]

        # Jika jumlah request lebih dari threshold, flag sebagai DoS
        if len(recent_requests) > dos_threshold:
            return f"DoS Attack Detected (Requests: {len(recent_requests)} in 1 minute)"
    except Exception as e:
        print(f"Error parsing log_time: {e}")

    # Jika tidak ada serangan yang terdeteksi
    return "Normal"


In [28]:
def parse_logs_with_attack_detection(input_path, output_path, pattern, row_limit):
    """
    Parsing log, mendeteksi pola serangan, dan menyimpan ke file CSV.
    """
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', newline='') as outfile:
        
        csv_writer = csv.writer(outfile)
        # Menulis header CSV
        csv_writer.writerow(['IP', 'User', 'Time', 'Method', 'URL', 'Status', 'Size', 'Referer', 'Agent', 'Hasil'])

        line_count = 0
        for line in infile:
            match = pattern.match(line)
            if match:
                # Deteksi pola serangan
                attack_result = detect_attack(
                    ip=match.group('ip'),
                    method=match.group('method'),
                    url=match.group('url'),
                    status=match.group('status'),
                    log_time=match.group('time')
                )
                
                # Menulis data ke CSV
                csv_writer.writerow([
                    match.group('ip'),
                    match.group('user'),
                    match.group('time'),
                    match.group('method'),
                    match.group('url'),
                    match.group('status'),
                    match.group('size'),
                    match.group('referer'),
                    match.group('agent'),
                    attack_result  # Tambahkan hasil deteksi pola serangan
                ])
                line_count += 1

            # Batasi jumlah baris yang diproses
            if line_count >= row_limit:
                break

        print(f'Parsing complete! {line_count} lines processed.')
        print(f'Output saved to {output_path}')


In [29]:
# File input (log) dan output (CSV)
input_file = 'new.log'  # Ubah sesuai file log Anda
output_file = 'Dataset.csv'

# Limit baris yang diproses
row_limit = 100000  


In [30]:
# Menjalankan parsing log dengan deteksi pola serangan
parse_logs_with_attack_detection(input_file, output_file, log_pattern, row_limit)


Parsing complete! 100000 lines processed.
Output saved to Dataset.csv
